In [5]:
import urllib.request
import json
import os
import ssl
import ipywidgets as widgets
from IPython.display import display


def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True)  # this line is needed if you use self-signed certificate in your scoring service.

# Widgets para la entrada de datos
column2_widget = widgets.IntText(description='Numero:')
peak_widget = widgets.Dropdown(options=['Everest', 'Annapurna II', 'Makalu', 'Ama Dablam', 'Cho Oyu', 'Manaslu', 'Lhotse', 'Kangchenjunga', 'Yalung Kang', 'Dhaulagiri I', 'Annapurna I Middle'],
                               description='Peak Name:')
year_widget = widgets.IntText(description='Year:')
season_widget = widgets.Dropdown(
    options=['Spring', 'Autumn', 'Summer', 'Winter'], description='Season:')
members_widget = widgets.IntText(description='Members:')
oxygen_widget = widgets.Dropdown(options=['Yes', 'No'], description='Oxygen Used:')


input_widgets = widgets.VBox([
    # column2_widget,
    peak_widget,
    year_widget,
    season_widget,
    members_widget,
    oxygen_widget

])

display(input_widgets)


def get_user_input():
    return {
        'Column2': column2_widget.value,
        'peak_name': peak_widget.value,
        'year': year_widget.value,
        'season': season_widget.value,
        'members': members_widget.value,
        'oxygen_used': oxygen_widget.value
    }


predict_button = widgets.Button(description='Predict')
output = widgets.Output()


def on_predict_button_clicked(b):
    input_data = get_user_input()

    # Map season to numerical index based on the options list
    season_mapping = {
        'Spring': 0,
        'Autumn': 1,
        'Summer': 2,
        'Winter': 3
    }

    # Estructuramos los datos en el formato esperado por el modelo
    data = {
        "input_data": {
            "columns": [
                'Column2',
                "year",
                "members",
                "oxygen_used",
                "season",
                'peak_name'
            ],
            "index": [0],
            "data": [
                [
                    input_data['peak_name'],
                    int(input_data['year']),
                    int(input_data['members']),
                    int(1 if input_data['oxygen_used'] == 'Yes' else 0),
                    season_mapping[input_data['season']],
                    int(input_data['Column2'])
                ]
            ]
        }
    }

    body = str.encode(json.dumps(data))

    url = 'https://himalaya-expeditions-dmzqx.eastus2.inference.ml.azure.com/score'
    # Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
    api_key = '6LQtcS5VCfQkTPnzvYvfGGTuMUH5hTzE'
    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")

    # The azureml-model-deployment header will force the request to go to a specific deployment.
    # Remove this header to have the request observe the endpoint traffic rules
    headers = {'Content-Type': 'application/json',
               'Authorization': ('Bearer '+api_key), 'azureml-model-deployment': 'expeditions83-1'}

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        result = response.read()
        print(result)
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the request ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))


predict_button.on_click(on_predict_button_clicked)
display(predict_button, output)


Button(description='Predict', style=ButtonStyle())

Output()

b'[0]'
b'[0]'
b'[0]'
b'[0]'
b'[0]'
b'[0]'
b'[0]'
b'[0]'
b'[0]'
b'[0]'
b'[0]'
b'[0]'
b'[0]'
